In [28]:
import numpy as np
import open3d as o3d
from open3d import JVisualizer
import pcl
import pcl.pcl_visualization
from src import utils
import random
from ipywidgets import interact_manual, IntSlider, BoundedIntText, FloatSlider, RadioButtons


class Struct3D():
    def __init__(self):
        self.x = []
        self.y = []
        self.z = []

        
def remove_points(cloud, indices):
    cloud_points = np.full((cloud.size, 3), cloud, dtype=np.float32)
    finalpoints = np.delete(cloud_points, indices, axis=0)
    return pcl.PointCloud(finalpoints)


def jupyter_visualize(points):
    print("-------VISUALIZING-------")
    vis = o3d.geometry.PointCloud()
    vis.points = o3d.utility.Vector3dVector(list(points[:, :3]))
    if points.shape[1] == 6:
        vis.colors = o3d.utility.Vector3dVector(list(points[:, 3:]))
    visualizer = JVisualizer()
    visualizer.add_geometry(vis)
    visualizer.show()

    
def segmentation(
    threshold=100,
    ksearch=10,
    distance_threshold=0.1,
    normal_distance_weight=0.005,
    max_iterations=500,
    eps_angle=np.pi/90.0,
    axis=(0, 0, 1),
    accept=False,
):
    global initial_cloud, cloud, planes, segmented_cloud
    x, y, z = axis
    visualize_points = np.empty((0, 6), dtype=np.float32)
    seg = utils.setup_segmenter(
        cloud,
        x, y, z,
        ksearch=ksearch,
        distance_threshold=distance_threshold,
        normal_distance_weight=normal_distance_weight,
        max_iterations=max_iterations,
        eps_angle=eps_angle
    )
    indices, coefficients = seg.segment()
    if len(indices) > threshold:
        inliers = utils.get(cloud, indices, utils.VisualizeType.ONLY_INLIERS)
        if accept:
            cloud = remove_points(cloud, indices)
            if x == 1:
                planes.x.append(coefficients)
                segmented_cloud.x.append(inliers)
            elif y == 1:
                planes.y.append(coefficients)
                segmented_cloud.y.append(inliers)
            else:
                planes.z.append(coefficients)
                segmented_cloud.z.append(inliers)
            color = [0.0, 1.0, 0.0]
        else:
            color = [1.0, 0.0, 0.0]
        visualize_points = np.append(
            visualize_points,
            utils.add_color_to_points(inliers, color),
            axis=0
        )
    for plane in segmented_cloud.x:
        visualize_points = np.append(
            visualize_points,
            utils.add_color_to_points(plane, [0.6, 0.6, 0.6]),
            axis=0
        )
    for plane in segmented_cloud.y:
        visualize_points = np.append(
            visualize_points,
            utils.add_color_to_points(plane, [0.6, 0.6, 0.6]),
            axis=0
        )
    for plane in segmented_cloud.z:
        visualize_points = np.append(
            visualize_points,
            utils.add_color_to_points(plane, [0.6, 0.6, 0.6]),
            axis=0
        )
    visualize_points = np.append(
        visualize_points,
        utils.add_color_to_points(initial_cloud, [0.0, 0.0, 0.0]),
        axis=0
    )
    utils.visualize(visualize_points)


def intersection(planes):
    corners = []

    for i in range(len(planes.x)):
        for j in range(len(planes.y)):
            A = [planes.z[0][:3], planes.x[i][:3], planes.y[j][:3]]
            B = [-planes.z[0][3], -planes.x[i][3], -planes.y[j][3]]
            corner = np.linalg.solve(A, B)
            # TODO is corner even a part of initial cloud?
            corners.append(list(corner))

    corners = np.asarray(corners)
    utils.visualize(corners)

    return corners


def get_net(corners, segmented_cloud):
    passed = list(range(len(corners)))
    keep = {}
    for i in passed:
        keep[str(i)] = [[], [], [], []]

    i = 0
    while len(passed) > 0:
        try:
            passed.remove(i)
            f = i
            for j in passed:
                diff_x = corners[i, 0] - corners[j, 0]
                diff_y = corners[i, 1] - corners[j, 1]
                if 0 < diff_x < 0.1:
                    keep[str(i)][0].append((abs(diff_x), j))
                    keep[str(j)][1].append((abs(diff_x), i))
                    f = j
                if 0 >= diff_x > -0.1:
                    keep[str(i)][1].append((abs(diff_x), j))
                    keep[str(j)][0].append((abs(diff_x), i))
                    f = j
                if 0 < diff_y < 0.1:
                    keep[str(i)][2].append((abs(diff_y), j))
                    keep[str(j)][3].append((abs(diff_y), i))
                    f = j
                if 0 >= diff_y > -0.1:
                    keep[str(i)][3].append((abs(diff_y), j))
                    keep[str(j)][2].append((abs(diff_y), i))
                    f = j
            i = f
        except ValueError as e:
            break
    
    print(corners[passed])
    print(keep)

    result = []
    for key, directions in keep.items():
        for d in directions:
            if len(d) == 0:
                continue
            min_diff = min(d)[1]
            if (int(key), min_diff) not in result and (
                min_diff, int(key)) not in result and edge_exists(
                    corners[int(key)], corners[min_diff], segmented_cloud):
                result.append((int(key), min_diff))
    return result


def edge_exists(point_a, point_b, segmented_cloud):
    if point_a[0] == point_b[0] and point_a[1] == point_b[1]:
        return False
    if abs(point_a[0] - point_b[0]) <= 0.1:
        middle = (point_a[1] + point_b[1]) / 2
        for p in segmented_cloud.x:
            p = np.asarray(p)
            together = np.where((abs(p[:, 1] - middle) <= 0.1) &
                                (abs(p[:, 0] - point_b[0]) <= 0.1) &
                                (abs(p[:, 0] - point_a[0]) <= 0.1))[0]
            if len(together) > 0:
                return True
    elif abs(point_a[1] - point_b[1]) <= 0.1:
        middle = (point_a[0] + point_b[0]) / 2
        for p in segmented_cloud.y:
            p = np.asarray(p)
            together = np.where((abs(p[:, 0] - middle) <= 0.1) &
                                (abs(p[:, 1] - point_b[1]) <= 0.1) &
                                (abs(p[:, 1] - point_a[1]) <= 0.1))[0]
            if len(together) > 0:
                return True

    return False


def get_polygon_indices(net):
    result = []

    while len(net) > 0:
        start_corner, next_corner = net[0]
        polygon = [start_corner]
        del net[0]

        while next_corner != start_corner:
            found = [(i, item) for i, item in enumerate(net) if next_corner in item]
            if len(found) == 0:
                break
            index, item = found[0]
            polygon.append(next_corner)
            next_corner = net[index][1 if item[0] == next_corner else 0]
            del net[index]

        polygon.append(start_corner)
        result.append(polygon)

    return result

FILENAME = "C:/Users/mbano/Documents/Faks/Projekt - Mobilna robotika/pcl-get-mesh/simulate_realsense_d435i/results/house.pcd"
LEAF_SIZE = 0.05

cloud = pcl.load(FILENAME)
vg = cloud.make_voxel_grid_filter()
vg.set_leaf_size(LEAF_SIZE, LEAF_SIZE, LEAF_SIZE)
cloud = vg.filter()

planes = Struct3D()
segmented_cloud = Struct3D()
initial_cloud = np.full((cloud.size, 3), cloud, dtype=np.float32)

segmentation_interact = interact_manual.options(manual_name="Segment planes")
segmentation_interact(
    segmentation,
    threshold=BoundedIntText(value=100, min=100, max=100_000, step=100),
    ksearch=IntSlider(value=10, min=10, max=30),
    distance_threshold=FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01),
    normal_distance_weight=FloatSlider(value=0.005, min=0.001, max=0.2, step=0.001, readout_format='.3f'),
    max_iterations=IntSlider(value=500, min=100, max=1500, step=100),
    eps_angle=FloatSlider(value=np.pi/90.0, min=np.pi/180.0, max=np.pi/6.0, step=np.pi/180.0),
    axis=RadioButtons(value=(0,0,1), options=[(1,0,0), (0,1,0), (0,0,1)]),
    accept=False,
)

def rest():
    global initial_cloud, cloud, plane, segmented_cloud
    corners = intersection(planes)
    net = get_net(corners, segmented_cloud)
    polygon_indices = get_polygon_indices(net)
    for p in polygon_indices:
        print(corners[p])

corners_interact = interact_manual.options(manual_name="Get corners")
corners_interact(rest)

interactive(children=(BoundedIntText(value=100, description='threshold', max=100000, min=100, step=100), IntSl…

interactive(children=(Button(description='Get corners', style=ButtonStyle()), Output()), _dom_classes=('widget…

<function __main__.rest()>